<h1> Project 1 </h3>

- this is practice project in practice too, but this one is graded
    - filename includes practice --> not graded and vice versa
- this one will be about line plots via pygal
    - but functions to extract data from the source (world data bank) will be written by me
    - to be useful regardless of the library
- assignments notes:
    - brown background code check is for practice, white official
    - so I will need gdpinfo dict to reuse to code (+ it will be used in upcoming projects) as a mapping/setup dict
    - problem 1: read CSV file as a nested dict
    - problem 2: list of GDP data suitable for XY plotting (list of tuples of year and value pair)
    - problem 3: dict mapping countries to XY plot (country: list of value pairs)
    - problem 4: create SVG image of XY plots for yearly GDP for SPECIFIED list of countries (use pygal here)
    - authors provided SVG file that should roughly the same like mine
    - btw they havent told us enough about pygal to do this on purpose --> use documentation (!)
- CAREFUL: some countries in the CSV have a comma in the name, so separator might be needed different for them
- dict gdpinfo might not make much sense now, but will later when working through the project
- template here: https://storage.googleapis.com/codeskulptor-isp/course4/isp_plot_template.py

In [147]:
################PROJECT PART###########################

import csv
import pygal

gdpinfo = {
    "gdpfile": "isp_gdp.csv",        # Name of the GDP CSV file
    "separator": ",",                # Separator character in CSV file
    "quote": '"',                    # Quote character in CSV file
    "min_year": 1960,                # Oldest year of GDP data in CSV file
    "max_year": 2015,                # Latest year of GDP data in CSV file
    "country_name": "Country Name",  # Country name field name
    "country_code": "Country Code"   # Country code field name
}

def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """
    with open(filename, "r") as csv_file:
      csvfile = csv.reader(csv_file, delimiter= separator, quotechar = quote)
      outer_dict = {}       #getting the output dict ready

      #IN THIS PART IM GETTING HEADERS TO BE USED IN THE INNER DICTS CREATIONS
      counter = 0
      for line in csvfile:
        counter = counter + 1
        if counter == 1:
          header_list = line
          break

      #IN THIS PART I NEED TO MAKE THE INNTER DICT CREATION LOOP
      counter = 0
      for line in csvfile:
        counter = counter + 1
        if counter == 1:
          pass             #skipping the first row here
        inner_dict = {}     #making the inner dict for each country
        key_value = line[header_list.index(keyfield)] #key_value (the mapping one) is taken via the index in the header
        for number, column in enumerate(line):          #taking columns of each line and their order
          inner_dict[header_list[number]] = column     #same counts of columns everywhere --> using enum index to make key:value pair
        outer_dict[key_value] = inner_dict              #adding the inner dict to the outer one
    return outer_dict

# print(read_csv_as_nested_dict("isp_gdp.csv", "Country Code", ",", '"'))


def build_plot_values(gdpinfo, gdpdata):
    """
    Inputs:
      gdpinfo - GDP data information dictionary
      gdpdata - A single country's GDP stored in a dictionary whose
                keys are strings indicating a year and whose values
                are strings indicating the country's corresponding GDP
                for that year.

    Output: 
      Returns a list of tuples of the form (year, GDP) for the years
      between "min_year" and "max_year", inclusive, from gdpinfo that
      exist in gdpdata.  The year will be an integer and the GDP will
      be a float.
    """
    list_of_tuples = []           #preparing the to-be-returned list
    for key in gdpdata.keys():    #getting keys of the country dict
       if key.isnumeric():        #checking whether its a pair about year/gdp
          if gdpinfo["min_year"] <= int(key) <= gdpinfo["max_year"] and gdpdata[key] != '':   #if its withing range and data is available
            value_tuple = (int(key), float(gdpdata[key]))     #adding the year and value as a tuple of 2 values ...
            list_of_tuples.append(value_tuple)                #...to the to be returned list
    return sorted(list_of_tuples)

# for i in build_plot_values(gdpinfo, read_csv_as_nested_dict("isp_gdp.csv", "Country Code", ",", '"').get("UKR")):
#    print(i)

def build_plot_dict(gdpinfo, country_list):
    """
    Inputs:
      gdpinfo      - GDP data information dictionary
      country_list - List of strings that are country names

    Output:
      Returns a dictionary whose keys are the country names in
      country_list and whose values are lists of XY plot values 
      computed from the CSV file described by gdpinfo.

      Countries from country_list that do not appear in the
      CSV file should still be in the output dictionary, but
      with an empty XY plot value list.
    """
    data = read_csv_as_nested_dict(gdpinfo["gdpfile"], gdpinfo["country_name"], gdpinfo["separator"], gdpinfo["quote"])
    #commented out option would be imo possible too
    plot_dict = {}  #preparing the dict to be returned
    for country in country_list:
      country_data = data.get(country, {})
      plot_dict[country] = build_plot_values(gdpinfo, country_data)
      #add a key/value pair of country and the plot values (via that function using the value from the nested dict via i key)
    return plot_dict

# dict_keys = read_csv_as_nested_dict(gdpinfo["gdpfile"], gdpinfo["country_name"], gdpinfo["separator"], gdpinfo["quote"]).keys()

# for i in build_plot_dict(gdpinfo, dict_keys):
#    print(i)
#    print(plot_dict.get(i))
# print(build_plot_dict(gdpinfo, dict_keys))

def render_xy_plot(gdpinfo, country_list, plot_file):
    """
    Inputs:
      gdpinfo      - GDP data information dictionary
      country_list - List of strings that are country names
      plot_file    - String that is the output plot file name

    Output:
      Returns None.

    Action:
      Creates an SVG image of an XY plot for the GDP data
      specified by info for the countries in country_list.
      The image will be stored in a file named by plot_file.
    """
    data_for_plotting = build_plot_dict(gdpinfo, country_list)  #via list of coutnries, get data
    scatter_chart = pygal.XY(stroke = False)  #initialize the plot
    scatter_chart.title = "Plot of GDP for selected countries 1960-2015"
    for i in data_for_plotting:   #for each country...
      scatter_chart.add(i, data_for_plotting.get(i))  #...add data to the plot
    scatter_chart._x_title = "Year"
    scatter_chart._y_title = "GDP in current US dollars"
    scatter_chart.render_to_file(plot_file)

country_list = ["United Kingdom", "United States"]
render_xy_plot(gdpinfo, country_list, "project_1_USA_UK.svg")
country_list = ["China"]
render_xy_plot(gdpinfo, country_list, "project_1_China.svg")
country_list = []
render_xy_plot(gdpinfo, country_list, "project_1_none.svg")
country_list = ["Czech Republic", "Slovak Republic", "Germany"]
render_xy_plot(gdpinfo, country_list, "project_1_CZ_SVK_GER.svg")